#Fase 1: Indexar legislação

In [ ]:
!pip install groq
!pip install -q pyserini==0.21.0
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_6

In [ ]:
import re
import json
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
import groq
from groq import Groq
import numpy as np
from tqdm import tqdm
from google.colab import userdata
import time
import pickle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = 'DEL5452.html'

with open(file_path, 'r', encoding = 'latin-1', errors = 'ignore') as file:
    clt_txt = file.read()

##Removendo artigos alterados

In [ ]:
pattern = r'<strike>.*?<\/strike>'
clt_txt = re.sub(pattern, '', clt_txt, flags = re.DOTALL)

## Selecionando artigos e parágrafos

In [ ]:
pattern = r'<a name="art.*?<\/font>'
paragraphs = re.findall(pattern, clt_txt, re.DOTALL)
len(paragraphs)

2877

##Limpeza das tags html e listagem dos artigos da CLT

In [ ]:
clean_paragraphs = []
agregated_text = ''
prev_title = 0
for i in paragraphs:
  title = re.findall('\d+', re.findall(r'name=\"art(.*?)\>', i)[0])[0]
  soup = BeautifulSoup(i, 'html.parser')
  texto = soup.get_text(separator=' ', strip=True)
  texto = re.sub(r'[\n\xa0\t]', '', texto)
  texto = re.sub(r'\((.*?)\)', '', texto)
  if prev_title == title:
    if texto != '':
      agregated_text += texto + '\n'
  else:
    clean_paragraphs.append({"contents": agregated_text, "title": prev_title})
    agregated_text = texto + '\n'
    prev_title = title

##Indexação para busca com BM25 pelo pyserini

In [ ]:
!mkdir data
!mkdir data/clt_indices

In [ ]:
f = open("data/clt_indices/contents.jsonl",'w')
id_finder = dict()

for i, doc in enumerate(clean_paragraphs):
    doc['id'] = i
    id_finder[doc['title']] = i
    if doc['contents'] != "":
        f.write(json.dumps(doc)+"\n")

In [ ]:
!python3 -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 1 -input data/clt_indices -index data/clt_index -storeRaw

pyserini.index is deprecated, please use pyserini.index.lucene.
2024-06-26 13:47:27,061 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2024-06-26 13:47:27,064 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2024-06-26 13:47:27,065 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2024-06-26 13:47:27,065 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: data/clt_indices
2024-06-26 13:47:27,066 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2024-06-26 13:47:27,067 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2024-06-26 13:47:27,067 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2024-06-26 13:47:27,067 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2

#Fase 2: Elencar perguntas e respostas


##Carregar perguntas do site da jusbrasil

In [ ]:
file_path = '50perguntas.txt'
with open(file_path, 'r', encoding = 'utf-8') as file:
  jus_p = file.read()

##Separação das perguntas e respostas

In [ ]:
perguntas = dict()
resposta = ''
pergunta = ''
for i in jus_p.split('\n\n'):
  if i[0].isdigit() and '-' in i:
    if resposta != '':
      perguntas[re.sub(r'\d+ - ','', pergunta)] = resposta
      pergunta = ''
      resposta = ''
    pergunta = i
  else:
    resposta += (i + '\n')
  [i for i in jus_p.split('\n\n') if i[0].isdigit() and '-' in i]

#Fase 3: Criar novas perguntas

In [ ]:
!python -m pip install --upgrade pip
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.2 MB/s eta 0:00:00


In [ ]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher('./data/clt_index')

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [ ]:
def query_prompt(query):
  '''
  Dada uma query, faz uma busca na CLT previamente indexada e retorna os 5 artigos mais relevantes.

  Input: query (str): A query a ser buscada na CLT

  Output: Uma string com os 5 artigos mais relevantes para a query, separados por duas quebras de linha.
  '''
  arts = []
  hits = searcher.search(query, k = 5)
  ask = "\n\nOs artigos utilizados são:\n\n"
  for idx, hit in enumerate(hits):
    hit = json.loads(hit.raw)
    ask += "{art}\n\n".format(idx = idx, art = hit['contents'])
    arts.append(hit['title'])
  return ask, arts

In [ ]:
client = Groq(api_key=userdata.get('groq.key'))
def send_message(message):
    '''
    Dada uma mensagem, manda pelo groq para o llama3-70b-8192 e retorna a resposta do LLM.

    Input: message (str): A mensagem a ser enviada ao LLM.

    Observações: Note que para minimizar as incertezas do processo e buscando maximizar a qualidade da resposta,
    estamos usando a temperatura 0 e top_p 1.

    Output: Uma string contendo a saída do Llama3
    '''
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": message}],
        model='llama3-70b-8192',
        temperature = 0,
        top_p = 1
    )
    return chat_completion.choices[0].message.content

In [ ]:
def create_question(article):
  '''
  Dado um artigo, este agente buscará formular um par pergunta - resposta.

  Input: article (str): O artigo da CLT em cima do qual se deseja formular uma pergunta.

  Observações: Note que para gerar a questão o agente usa um exemplo das perguntas da Jusbrasil
  como one-shot.

  Output: Uma string contendo a saída do agente, que idealmente será um par pergunta - resposta
  baseado no artigo de entrada. O padrão é Pergunta: --- Resposta: ---.
  '''

  return send_message('''Você está escrevendo perguntas para um concurso de direito do trabalho, seu papel é utilizar um artigo para gerar uma questão na qual o profissional dessa área se depare no futuro. Segue abaixo um exemplo para mostrar o que deve ser feito:

Artigo: Art. 473 - O empregado poderá deixar de comparecer ao serviço sem prejuízo do salário: \n\nII - até 3  dias consecutivos, em virtude de casamento; \nIII - por 5  dias consecutivos, em caso de nascimento de filho, de adoção ou de guarda compartilhada; \nIV - por um dia, em cada 12  meses de trabalho, em caso de doação voluntária desangue devidamente comprovada; \nV - até 2  dias consecutivos ou não, para o fim de se alistar eleitor, nos têrmosda lei respectiva. \nVI - no período de tempo em que tiver de cumprir as exigências doServiço Militar referidas na letra "c" do art. 65 da Lei nº 4.375, de 17 deagosto de 1964 . \n\n\nIX - pelo tempo que se fizer necessário, quando, na qualidade derepresentante de entidade sindical, estiver participando de reunião oficial de organismointernacional do qual o Brasil seja membro. \nX - até 2  dias para acompanhar consultas médicas e exames complementares durante o período de gravidez de sua esposa ou companheira;  X - pelo tempo necessário para acompanhar sua esposa ou companheira em até 6  consultas médicas, ou em exames complementares, durante o período de gravidez;  XI - por 1  dia por ano para acompanhar filho de até 6  anos em consulta médica.  XII - até 3  dias, em cada 12  meses de trabalho, em caso de realização de exames preventivos de câncer devidamente comprovada. \nParágrafo único. O prazo a que se refere o inciso III do caput deste artigo será contado a partir da data de nascimento do filho.
Pergunta: CASEI, POSSO FALTAR AO SERVIÇO SEM TER O SALÁRIO DESCONTADO?
Resposta: Sim, o trabalhador que vai se casar tem direito a três dias seguidos de folga, sem que tenha nenhum tipo de desconto no salário.

Agora faça uma pergunta e uma resposta usando a mesma formatação, embasando-se apenas no artigo a seguir:
Artigo: {artigo}
'''.format(artigo = article))


In [ ]:
def judge_question(question):
  '''
  Dada uma questão, este agente avalia se a pergunta é válida ou inválida.

  Input: question (str): Um par pergunta - resposta.

  Observações: Esse agente também usa uma das questões que julguei inválidas como exemplo para one-shot.

  Output: Uma string indicando se a pergunta é válida ou não no formato: Aprovada: (Sim/Não)
  '''

  return send_message('''Você está montando um concurso de direito do trabalho, e está recebendo sugestões de perguntas para aplicar na sua prova. Seu papel é avaliar se a pergunta faz
sentido sem dar informações adicionais ao participante... Julgue o seguinte par pergunta e resposta:

Descarte as perguntas que fazem referência a informações que não possam ser recuperadas, exemplo: QUAL É O OBJETIVO PRINCIPAL DESTA CONSOLIDAÇÃO? deve ser descartada, pois como saberemos o que "esta consolidação" faz referência?

Pergunta: {question}

Responda de acordo com a formatação abaixo:

Aprovada: Sim / Não
'''.format(question = question))

In [ ]:
added_questions = []
for idx, i in enumerate(tqdm(clean_paragraphs)):
  if i['contents'] != '':
    #Cria a questão
    q = create_question(i['contents'])
    #Avalia a questão
    veredict = judge_question(q)
    #Se aprovada
    if 'não' not in veredict[:15].lower():
      #Salva questão e artigo que a originou
      added_questions.append({'qa': q, 'passagens': i['contents']})

      '''
      Opcional para colab, evitando que o ambiente desconecte perdendo o progresso
      Descomente essa seção se optar.

      if(idx % 25 == 0 and idx != 0):
        print('writing q\'s at idx ', idx)
        file_path = 'new_qa.pkl'
        with open(file_path, 'wb') as file:
            pickle.dump(added_questions, file)
      '''

  3%|▎         | 26/844 [03:43<2:24:19, 10.59s/it]

writing q's at idx  25


  6%|▌         | 51/844 [08:33<3:10:18, 14.40s/it]

writing q's at idx  50


  9%|▉         | 76/844 [13:13<2:24:20, 11.28s/it]

...

 95%|█████████▍| 801/844 [2:29:21<07:44, 10.81s/it]

writing q's at idx  800


 98%|█████████▊| 826/844 [2:34:37<03:32, 11.80s/it]

writing q's at idx  825


100%|██████████| 844/844 [2:37:45<00:00, 11.22s/it]


##Opcional: Leitura das perguntas salvas

In [ ]:
'''
file_path = 'new_qa.pkl'

with open(file_path, 'rb') as file:
    added_questions = pickle.load(file)

'''

##Separação das perguntas, respostas e passagens

Eles devem seguir o formato de dicionário
```
qap['pergunta'] = {'resposta': r,
                  'passagens': ps}
```



In [ ]:
qap = dict()
for i in added_questions:
  qap[re.findall("Pergunta: .+", i['qa'])[0][10:]] = {'resposta': re.findall("Resposta: .+", i['qa'])[0][10:], 'passagens': i['passagens']}

#Fase 4: Enriquecer perguntas com passagens da legislação

In [ ]:
map_art = dict()

for i in clean_paragraphs:
  if i['contents'] != '':
    map_art[i['title']] = i['contents']

def get_article(art_number):
  '''
  Dado o número de um artigo devolve ele

  Input: art_number (str/int): O artigo que se deseja

  Output: Ou o artigo, ou uma string vazia caso não encontre o artigo.
  '''
  if art_number not in map_art.keys():
    return ''
  return map_art[str(art_number)]

In [ ]:
def generate_HyDE_document(question, answer):
  '''
  Esse agente é responsável por criar o documento hipotético seguindo a estratégia de retrieval HyDE.

  link: https://arxiv.org/abs/2212.10496

  Input: question (str): A pergunta a ser respondida.
         answer (str): A resposta da pergunta.

  Observação: No artigo do HyDE não se tem acesso à resposta na hora da geração do documento, mas como
              no caso desse projeto temos acesso, optei por utilizá-lo para melhorar a qualidade do documento.

  Output: Um artigo forjado seguindo a estratégia de retrieval HyDE, para melhorar a resposta do buscador.
  '''
  return send_message('''Você está procurando na legislação um artigo que responda uma pergunta.

Escreva um artigo hipotético que, caso estivesse escrito, responderia a pergunta: {question}

Resposta: {answer}

Agora escreva um esboço do artigo que está procurando.
'''.format(question = question, answer = answer))

In [ ]:
def search_aditional_articles(answer, articles, art_list):
  '''
  Esse agente é responsável por procurar artigos adicionais que possam ser úteis

  Input: answer (str): A resposta da pergunta.
         articles (str): Uma lista dos artigos encontrados por retrieval.
         art_list (list[(int/str)]): Uma lista com o número dos artigos utilizados.

  Observação: A ideia desse agente é buscar por outros artigos que são mencionados, por exemplo
  o artigo 866 da CLT faz referência ao artigo 860. Esse agente deve pontuar a necessidade de buscar pelo artigo 860.

  Output: Uma string contendo os novos artigos buscados. (No máximo 3).
  '''

  m = send_message('''Você está procurando na legislação artigos que respondam uma pergunta.
Seu papel é assegurar que não precisa de informações de outros artigos, e, caso necessário procurar mais artigos com sua ferramenta: Procure:

Sua ação DEVE SEGUIR O PADRÃO: Procure: número do artigo.

Por exemplo, se quero procurar os artigos 12 e 55 a resposta DEVE ser separada por linhas:

Procure: 12
Procure: 55

Dada a resposta: {answer}

E dados os artigos: {articles}

Defina se há artigos que não foram citados e siga o padrão da sua ação.

Se os artigos já forem suficientes a resposta DEVE ser:

Procure: -1
'''.format(answer = answer, articles = articles))
  search_list = [i[9:] for i in re.findall(r'Procure: \d+', m)]
  return '\n\n'.join([get_article(i) for i in search_list if i not in art_list][:3])


In [ ]:
def discard_articles(question, articles, answer):
  '''
  Esse agente é responsável por descartar artigos que não sejam relevantes para responder a pergunta.

  Input: question (str): A pergunta.
         articles (str): Uma lista dos artigos encontrados por retrieval.
         answer (str): A resposta da pergunta.

  Output: Uma string contendo apenas os artigos úteis para responder a pergunta.
  '''

  return send_message('''Você está buscando na legislação apenas as informações relevantes para responder uma pergunta, mas está encontrando varias informações que deviam a sua atenção.

Descarte dos artigos à seguir aqueles que você não julgar extremamente relevantes para responder a seguinte pergunta: {question}

{articles}

Tente remover as informações que não sejam relevantes para a pergunta. Não altere os artigos que foram mantidos.

A resposta foi: {answer}

Não faça comentários sobre sua seleção, sua resposta deve ser apenas os artigos selecionados.
'''.format(question = question, articles = articles, answer = answer))

In [ ]:
inc_questions = dict()

In [ ]:
for idx, q in enumerate(tqdm(perguntas)):
  if q not in inc_questions.keys():

    #gera documento hipotético
    hyde = generate_HyDE_document(q, perguntas[q])

    #busca
    passagens, art_list = query_prompt(hyde)

    #search for more
    passagens += search_aditional_articles(perguntas[q], passagens, art_list)

    #descarta não relevantes
    base = discard_articles(q, passagens, perguntas[q])

    #salva
    inc_questions[q] = {'resposta': perguntas[q], 'passagens': base}

    '''
    Opcional para colab, evitando que o ambiente desconecte perdendo o progresso
    Descomente essa seção se optar.

    if (idx % 3 == 0) and idx != 0:
      print('writing q\'s at idx ', idx)
      file_path = 'jusss_qa.pkl'
      with open(file_path, 'wb') as file:
          pickle.dump(inc_questions, file)
    '''

 39%|███▉      | 19/49 [04:14<09:03, 18.11s/it]

writing q's at idx  18


 45%|████▍     | 22/49 [07:56<17:42, 39.36s/it]

writing q's at idx  21


 51%|█████     | 25/49 [13:02<29:17, 73.25s/it]

...

writing q's at idx  42


 94%|█████████▍| 46/49 [43:14<04:02, 80.94s/it]

writing q's at idx  45


100%|██████████| 49/49 [47:22<00:00, 58.02s/it]

writing q's at idx  48


#Fase 5: Avaliação do dataset por RAGAS

In [ ]:
#Escolher 44 questões dentro das 570 geradas
random_array = np.random.randint(0, len(qap), 44)

In [ ]:
for i in inc_questions:
  inc_questions[list(qap.keys())[i]] = qap[list(qap.keys())[i]]


##Salvar primeira bateria do dataset

In [ ]:
file_path = 'poc_qa.pkl'
with open(file_path, 'wb') as file:
  pickle.dump(inc_questions, file)

In [ ]:
file_path = 'poc_qa.pkl'

with open(file_path, 'rb') as file:
    inc_questions = pickle.load(file)

##Faithfulness

In [ ]:
def get_faithfulness(question, ans_pass):
  '''
  Calcula faithfulness de acordo com a avaliação do RAGAS.

  Input: question (str): A pergunta.
         ans_pass (dict): Um dicionário com a resposta e a passagem do artigo.

  Output: faithfulness score (float)
  '''
  answer = ans_pass['resposta']
  context = ans_pass['passagens']

  prompt_1 = '''Dada uma questão e uma resposta, separe a resposta nas afirmações feitas.
Questão: {question}
Resposta: {answer}

Use o seguinte formato para sua resposta:

afirmação: [afirmação 1]
...
afirmação: [afirmação n]
'''.format(question = question, answer = answer)
  statements = send_message(prompt_1)

  prompt_2 = '''Considerando o contexto:

  {context}
Determine se as afirmações feitas abaixo são suportadas pelo contexto. Explique brevemente antes de dar um veredito (Sim/Não). Dê um veredito para cada afirmação no seguinte formato:

afirmação - [veredito] - explicação

as afirmações são

{statements}
'''.format(context=context, statements=statements)
  fth_ans = send_message(prompt_2)

  print(prompt_2)
  print(fth_ans)
  ver_list = re.findall(r'-.+-', fth_ans)

  positive = len([i for i in ver_list if 'sim' in i.lower()])
  negative = len([i for i in ver_list if 'não' in i.lower()])
  return positive / (positive + negative)


In [ ]:
for idx, i in enumerate(tqdm(inc_questions)):
  faithfullnss_list.append(get_faithfulness(i, inc_questions[i]))

100%|██████████| 92/92 [06:24<00:00,  4.17s/it]


##Salvar

In [ ]:
file_path = 'faithfullnss.pkl'
with open(file_path, 'wb') as file:
  pickle.dump(faithfullnss_list, file)

In [ ]:
file_path = 'faithfullnss.pkl'
with open(file_path, 'rb') as file:
    faithfullnss_list = pickle.load(file)

##Cálculo do faithfulness médio

In [ ]:
np.mean(faithfullnss_list)

0.6631211180124224

##Answer relevance

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('aspire/acge_text_embedding')

def get_sentence_similarity(a, b, model):
  '''
  Calcula similaridade entre embeddings de duas sentenças.

  Input: a (str): Sentença a.
         b (str): Sentença b.
         model (SentenceTransformer): Um modelo sentence transformer com saída em embeddings.

  Output: sentence similarity (float)
  '''
  sentences = [a, b]
  embeddings_1 = model.encode(sentences, normalize_embeddings=True)
  embeddings_2 = model.encode(sentences, normalize_embeddings=True)
  similarity = embeddings_1 @ embeddings_2.T
  return similarity[0, 1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/33.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

In [ ]:
def get_ans_relevance(question, ans_pass, model):
  '''
  Calcula answer relevance de acordo com a avaliação do RAGAS.

  Input: question (str): A pergunta.
         ans_pass (dict): Um dicionário com a resposta e a passagem do artigo.
         model (SentenceTransformer): Um modelo sentence transformer com saída em embeddings.

  Output: answer relevance (float)
  '''
  answer = ans_pass['resposta']
  context = ans_pass['passagens']

  prompt_1 = '''Gere de duas a quatro possíveis perguntas para dada resposta.
Resposta: {answer}

Use o seguinte formato para sua resposta:

Pergunta: [pergunta 1]
...
Pergunta: [pergunta n]
'''.format(answer=answer)
  statements = send_message(prompt_1)

  q_pattern = re.compile(r'Pergunta: .+')
  q_list = q_pattern.findall(statements)
  similarities = []
  for q in q_list:
    similarities.append(get_sentence_similarity(q[10:], question, model))

  return sum(similarities)/len(similarities)

In [ ]:
answer_relevance_list = []

In [ ]:
for idx, i in enumerate(tqdm(inc_questions)):
  answer_relevance_list.append(get_ans_relevance(i, inc_questions[i], model))

100%|██████████| 92/92 [19:55<00:00, 13.00s/it]


##Calcula answer relevance médio

In [ ]:
np.mean(answer_relevance_list)

0.7773634640742904

## Salva

In [ ]:
file_path = 'ans_rel.pkl'
with open(file_path, 'wb') as file:
  pickle.dump(answer_relevance_list, file)

##Context relevance

In [ ]:
def get_con_relevance(question, ans_pass):
  '''
  Calcula context relevance de acordo com a avaliação do RAGAS.

  Input: question (str): A pergunta.
         ans_pass (dict): Um dicionário com a resposta e a passagem do artigo.

  Output: context relevance (float)
  '''
  answer = ans_pass['resposta']
  context = ans_pass['passagens']
  p = ''
  for idx, i in enumerate(context.split('\n\n'), 1):
    p += "Artigo {idx}: {i}\n".format(i=i, idx=idx)
  prompt_1 = '''Dados os artigos de contexto:

{p}

Marque para cada artigo um veredito: **sim** se ele for relevante para responder a pergunta, e **não** se não for relevante para responder a seguinte pergunta:

{question}

Só marque como **não** as sentenças que forem irrelevantes.

'''.format(p=p, question=question)
  statements = send_message(prompt_1)

  s_pattern = re.compile(r'\*\*.+')
  s_list = s_pattern.findall(statements)

  return len([i for i in s_list if 'sim' in i.lower()])/(len([i for i in s_list if 'sim' in i.lower()]) + len([i for i in s_list if 'não' in i.lower()]))

In [ ]:
context_relevance_list = []

In [ ]:
for idx, i in enumerate(tqdm(inc_questions)):
  context_relevance_list.append(get_con_relevance(i, inc_questions[i]))

100%|██████████| 92/92 [05:25<00:00,  3.53s/it]


## Calcula context relevance médio

In [ ]:
np.mean(context_relevance_list)

0.6248080421993466

## Salva

In [ ]:
file_path = 'con_rel.pkl'
with open(file_path, 'wb') as file:
  pickle.dump(context_relevance_list, file)

#Análises adicionais

##Análises faithfulness

In [ ]:
jus_list = []
sint_list = []
for idx, i in enumerate(inc_questions):
  if i in perguntas.keys():
    jus_list.append(idx)
  else:
    sint_list.append(idx)

In [ ]:
#Calcula diferença de faithfulness entre sintético e obtido na jusbrasil
print('Média faithfulness jus: ', np.mean(np.array(faithfullnss_list)[np.array(jus_list)]))
print('Média faithfulness sintético: ', np.mean(np.array(faithfullnss_list)[np.array(sint_list)]))

Média faithfulness jus:  0.48115079365079355
Média faithfulness sintético:  0.8616341991341993


In [ ]:
x = np.array(faithfullnss_list)[np.array(jus_list)]
np.where(x == x.min())

(array([ 1,  3,  4, 10, 20, 25, 26, 29, 32, 33, 37]),)

In [ ]:
list(inc_questions.keys())[jus_list[np.argmin(np.array(faithfullnss_list)[np.array(jus_list)])]]

'O QUE FAZER SE ATRASAREM MEU SALÁRIO?'

In [ ]:
list(inc_questions.values())[jus_list[np.argmin(np.array(faithfullnss_list)[np.array(jus_list)])]]

{'resposta': 'O empregado que tem ou teve seu salário atrasado de forma frequente por vários meses, pode solicitar na justiça a rescisão indireta do seu contrato de trabalho, ou seja, o rompimento do contrato com o pagamento de todas as verbas que teria direito em caso de demissão sem justa causa.\nAtraso de menos de 20 dias incide multa de 10% sobre o salário devido.\nAtraso com mais de 20 dias incide a multa de 10% sobre o valor do saldo devedor, mais 5% sobre todos os dias úteis após o 20º dia.\n',
 'passagens': 'Art. 477. Na extinção do contrato de trabalho, o empregador deverá proceder à anotação na Carteira de Trabalho e Previdência Social, comunicar a dispensa aos órgãos competentes e realizar o pagamento das verbas rescisórias no prazo e na forma estabelecidos neste artigo. \n§ 2º - O instrumento de rescisão ou recibo de quitação, qualquer que seja a causa ou forma de dissolução do contrato, deve ter especificada a natureza de cada parcela paga ao empregado e discriminado o seu

##caso 2

In [ ]:
list(inc_questions.keys())[jus_list[3]]

'TRABALHEI FERIADO E NÃO ME COMPENSARAM COM FOLGA, COMO DEVO RECEBER?'

In [ ]:
print(list(inc_questions.values())[jus_list[3]]['passagens'])

Art. 59. A duração diária do trabalho poderá ser acrescida de horas extras, em número não excedente de duas, por acordo individual, convenção coletiva ou acordo coletivo de trabalho.  
§ 1 o A remuneração da hora extra será, pelo menos, 50%  superior à da hora normal.  
§2 o Poderá    ser dispensado o acréscimo de salário se, por força de acordo ou convenção coletiva    de trabalho, o excesso de horas em um dia for compensado pela correspondente diminuição    em outro dia, de maneira que não exceda, no período máximo de um ano, à soma das    jornadas semanais de trabalho previstas, nem seja ultrapassado o limite máximo de dez    horas diárias.

Art. 59-A. Em exceção ao disposto no art. 59 desta Consolidação , é facultado às partes, mediante acordo individual escrito, convenção coletiva ou acordo coletivo de trabalho, estabelecer horário de trabalho de doze horas seguidas por trinta e seis horas ininterruptas de descanso, observados ou indenizados os intervalos para repouso e alimentaçã

##Questões sintéticas

In [ ]:
x = np.array(faithfullnss_list)[np.array(sint_list)]
np.where(x == x.min())

(array([23]),)

In [ ]:
faithfullnss_list[sint_list[23]]

0.42857142857142855

In [ ]:
list(inc_questions.keys())[sint_list[23]]

'POSSO OBTER UMA CARTEIRA DE TRABALHO E PREVIDÊNCIA SOCIAL PARA USO DE QUÍMICOS SEM TER UM DIPLOMA DE QUÍMICO?'

In [ ]:
print(list(inc_questions.values())[sint_list[23]])

{'resposta': 'Não, para obter uma Carteira de Trabalho e Previdência Social para uso de químicos, é necessário ter um diploma de químico, químico industrial, químico industrial agrícola ou engenheiro químico, expedido por escola superior oficial ou oficializada, além de outros documentos comprovantes.',
 'passagens': '§ 1º - A requisição de Carteira de Trabalho e Previdência Social para uso dosquímicos, além do disposto no capítulo "Da Identificação Profissional",somente será processada mediante apresentação dos seguintes documentos que provem:\na)ser o requerente brasileiro, nato ou naturalizado, ou estrangeiro;\nb) estar, se forbrasileiro, de posse dos direitos civis e políticos;\nc) ter diploma dequímico, químico industrial, químico industrial agrícola ou engenheiro químico,expedido por escola superior oficial ou oficializada;\nd) ter, sediplomado no estrangeiro, o respectivo diploma revalidado nos termos da lei;\ne) haver, o quefor brasileiro naturalizado, prestado serviço militar 

In [ ]:
print(list(inc_questions.values())[sint_list[23]]['passagens'])

§ 1º - A requisição de Carteira de Trabalho e Previdência Social para uso dosquímicos, além do disposto no capítulo "Da Identificação Profissional",somente será processada mediante apresentação dos seguintes documentos que provem:
a)ser o requerente brasileiro, nato ou naturalizado, ou estrangeiro;
b) estar, se forbrasileiro, de posse dos direitos civis e políticos;
c) ter diploma dequímico, químico industrial, químico industrial agrícola ou engenheiro químico,expedido por escola superior oficial ou oficializada;
d) ter, sediplomado no estrangeiro, o respectivo diploma revalidado nos termos da lei;
e) haver, o quefor brasileiro naturalizado, prestado serviço militar no Brasil;
f) achar-se oestrangeiro, ao ser promulgada a Constituição de 1934 , exercendo legitimamente, naRepública, a profissão de químico, ou concorrer a seu favor a existência dereciprocidade internacional, admitida em lei, para o reconhecimento dos diplomas dessaespecialidade.
b) do certificadoou atestado comprobatório

##Limpeza das perguntas com faithfulness = 0

In [ ]:
x = np.array(faithfullnss_list)[np.array(jus_list)]
print('Média faithfulness jus: ', np.mean(x[np.where(x != 0)]))

Média faithfulness jus:  0.6241956241956241


In [115]:
x = np.array(faithfullnss_list)
for i in list(np.where(x == 0)[0])[::-1]:
  del inc_questions[list(inc_questions.keys())[i]]
  faithfullnss_list.pop(i)
  answer_relevance_list.pop(i)
  context_relevance_list.pop(i)

In [116]:
jus_list = []
sint_list = []
for idx, i in enumerate(inc_questions):
  if i in perguntas.keys():
    jus_list.append(idx)
  else:
    sint_list.append(idx)

In [118]:
print("Perguntas jusbrasil:", len(jus_list))
print("Perguntas sintéticas:", len(sint_list))

Perguntas jusbrasil: 37
Perguntas sintéticas: 43


In [120]:
print("Faithfulness: ", np.mean(faithfullnss_list))
print("Answer relevance: ", np.mean(answer_relevance_list))
print("Context relevance: ", np.mean(context_relevance_list))

Faithfulness:  0.7625892857142856
Answer relevance:  0.7787762671709061
Context relevance:  0.6924875818625819


In [129]:
min(answer_relevance_list)

0.5262129604816437

In [123]:
list(inc_questions.keys())[np.argmin(answer_relevance_list)]

'POSSO RECEBER MINHA CTPS EM FORMATO DIGITAL?'

In [126]:
print("resposta: ", list(inc_questions.values())[np.argmin(answer_relevance_list)]['resposta'])
print("passagem: ", list(inc_questions.values())[np.argmin(answer_relevance_list)]['passagens'])

resposta:  Sim, a emissão da CTPS em formato eletrônico é privilegiada, ou seja, é o formato preferencial para a emissão da carteira de trabalho.
passagem:  
Art. 15. Os procedimentos para emissão da CTPS ao interessado serão estabelecidos pelo Ministério da Economia em regulamento próprio, privilegiada a emissão em formato eletrônico. 



In [136]:
#Acha indices de context_relevance igual a 0
x = np.array(context_relevance_list)
np.where(x == x.min())[0][::-1]

array([35, 34, 16,  2])

In [131]:
list(inc_questions.keys())[np.argmin(context_relevance_list)]

'O QUE SIGNIFICA SALDO DE SALÁRIO?'

In [132]:
print("resposta: ", list(inc_questions.values())[np.argmin(context_relevance_list)]['resposta'])
print("passagem: ", list(inc_questions.values())[np.argmin(context_relevance_list)]['passagens'])

resposta:  Saldo de salário são os dias do mês trabalhado, ou seja, se o empregado trabalhou até o dia 12 de março, seu saldo de salário será de 12 dias. É devido em qualquer tipo de rescisão do contrato.

passagem:  Art. 477. Na extinção do contrato de trabalho, o empregador deverá proceder à anotação na Carteira de Trabalho e Previdência Social, comunicar a dispensa aos órgãos competentes e realizar o pagamento das verbas rescisórias no prazo e na forma estabelecidos neste artigo.

§ 2º - O instrumento de rescisão ou recibo de quitação, qualquer que seja a causa ou forma de dissolução do contrato, deve ter especificada a natureza de cada parcela paga ao empregado e discriminado o seu valor, sendo válida a quitação, apenas, relativamente às mesmas parcelas.

Art. 478 - A indenização devida pela rescisão de contrato por prazo indeterminado será de 1 mês de remuneração por ano de serviço efetivo, ou por ano e fração igual ou superior a 6 meses.

Art. 487 - Não havendo prazo estipulado, 

In [134]:
list(inc_questions.keys())[16]

'EMPREGADA DOMÉSTICA TEM DIREITO ÀS FÉRIAS?'

In [135]:
print("resposta: ", list(inc_questions.values())[16]['resposta'])
print("passagem: ", list(inc_questions.values())[16]['passagens'])

resposta:  Sim, contudo só pode parcelar as férias em 2 períodos, um deles de no mínimo 14 dias.

passagem:  Art. 134 - As férias serão concedidas por ato do empregador, em um só período, nos 12 meses subseqüentes à data em que o empregado tiver adquirido o direito.
§ 1 o Desde que haja concordância do empregado, as férias poderão ser usufruídas em até três períodos, sendo que um deles não poderá ser inferior a quatorze dias corridos e os demais não poderão ser inferiores a cinco dias corridos, cada um.

Art. 139 - Poderão ser concedidas férias coletivas a todos os empregados de uma empresa ou de determinados estabelecimentos ou setores da empresa.
§ 1º - As férias poderão ser gozadas em 2 períodos anuais desde que nenhum deles seja inferior a 10 dias corridos.


##Limpeza com context relevance = 0

In [137]:
x = np.array(context_relevance_list)
for i in list(np.where(x == 0)[0])[::-1]:
  del inc_questions[list(inc_questions.keys())[i]]
  faithfullnss_list.pop(i)
  answer_relevance_list.pop(i)
  context_relevance_list.pop(i)

In [138]:
jus_list = []
sint_list = []
for idx, i in enumerate(inc_questions):
  if i in perguntas.keys():
    jus_list.append(idx)
  else:
    sint_list.append(idx)

In [139]:
print("Perguntas jusbrasil:", len(jus_list))
print("Perguntas sintéticas:", len(sint_list))

Perguntas jusbrasil: 33
Perguntas sintéticas: 43


In [140]:
print("Faithfulness: ", np.mean(faithfullnss_list))
print("Answer relevance: ", np.mean(answer_relevance_list))
print("Context relevance: ", np.mean(context_relevance_list))

Faithfulness:  0.7698308270676691
Answer relevance:  0.7851408414150539
Context relevance:  0.7289342966974546


In [141]:
print('Média faithfulness jus: ', np.mean(np.array(faithfullnss_list)[np.array(jus_list)]))
print('Média faithfulness sintético: ', np.mean(np.array(faithfullnss_list)[np.array(sint_list)]))

Média faithfulness jus:  0.624098124098124
Média faithfulness sintético:  0.881672203765227


In [142]:
print('Média answer relevance jus: ', np.mean(np.array(answer_relevance_list)[np.array(jus_list)]))
print('Média answer relevance sintético: ', np.mean(np.array(answer_relevance_list)[np.array(sint_list)]))

Média faithfulness jus:  0.7572978071191094
Média faithfulness sintético:  0.8065087514561277


In [143]:
print('Média context relevance jus: ', np.mean(np.array(context_relevance_list)[np.array(jus_list)]))
print('Média context relevance sintético: ', np.mean(np.array(context_relevance_list)[np.array(sint_list)]))

Média context relevance jus:  0.6040404040404039
Média context relevance sintético:  0.8247830980389121


In [144]:
file_path = 'dataset_limpo.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(inc_questions, file)